In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
class Node:
    #defining the class of nodes for this case.
    def __init__(self, name, var, thres, res):
        self.id = name #the node number
        self.var = var #the feature on which we split
        self.thres = thres #the split threshold
        self.c1 = None
        self.c2 = None
        self.res = res

    def __str__(self):
        return f"Node {self.id}: Split on {self.var} with threshold {self.thres} or return result {self.res} \nLeft Child of {self.id}: {self.c1} \nRight Child of {self.id}: {self.c2}"
    
    #need to establish children methodologies
    def insert_left_child(self, data, id):
        #self.c1 = Node(new_var, new_thres, new_res)
        self.c1 = MakeNode(data, id)
    
    def insert_right_child(self, data, id):
        self.c2 = MakeNode(data, id)

    def depth(self):
        #get the depth of the tree
        #base case: leaf
        if self.c1 == None and self.c2 == None:
            return 1
        #recursive case: get depth of both subtrees and pick larger
        if self.c1 != None:
            d1 = self.c1.depth()
        if self.c2 != None:
            d2 = self.c2.depth()
        if d2 > d1:
            return d2
        else:
            return d1

In [24]:
df = pd.read_csv("toxicity-2/data.csv")
df

,MATS3v,nHBint10,MATS3s,MATS3p,nHBDon_Lipinski,minHBint8,MATS3e,MATS3c,minHBint2,MATS3m,...,WTPT-4,WTPT-5,ETA_EtaP_L,ETA_EtaP_F,ETA_EtaP_B,nT5Ring,SHdNH,ETA_dEpsilon_C,MDEO-22,Class
0,0.0908,0,0.0075,0.0173,0,0.0000,-0.0436,0.0409,0.0000,0.1368,...,0.0000,0.0000,0.1780,1.5488,0.0088,0,0.0,-0.0868,0.00,NonToxic
1,0.0213,0,0.1144,-0.0410,0,0.0000,0.1231,-0.0316,0.0000,0.1318,...,8.8660,19.3525,0.1739,1.3718,0.0048,2,0.0,-0.0810,0.25,NonToxic
2,0.0018,0,-0.0156,-0.0765,2,0.0000,-0.1138,-0.1791,0.0000,0.0615,...,5.2267,27.8796,0.1688,1.4395,0.0116,2,0.0,-0.1004,0.00,NonToxic
3,-0.0251,0,-0.0064,-0.0894,3,0.0000,-0.0747,-0.1151,0.0000,0.0361,...,7.7896,24.7336,0.1702,1.4654,0.0133,2,0.0,-0.1010,0.00,NonToxic
4,0.0135,0,0.0424,-0.0353,0,0.0000,-0.0638,0.0307,0.0000,0.0306,...,12.3240,19.7486,0.1789,1.4495,0.0120,2,0.0,-0.1071,0.00,NonToxic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,-0.0960,0,-0.0478,-0.0840,2,0.0000,-0.0739,-0.2315,1.5660,-0.1133,...,2.5690,12.0174,0.1648,0.9710,0.0049,1,0.0,-0.0952,0.00,NonToxic
167,-0.0064,1,-0.1222,0.0013,1,0.0000,-0.1873,-0.2181,5.5404,-0.0757,...,10.7860,6.4871,0.1805,1.2298,0.0127,1,0.0,-0.0860,0.00,NonToxic
168,0.0096,2,-0.1846,0.0058,1,0.0000,-0.1293,-0.0979,5.3976,0.0409,...,4.9930,19.2864,0.2089,1.1245,0.0093,1,0.0,-0.0927,0.00,NonToxic
169,-0.0736,2,-0.1267,-0.0345,2,0.5346,-0.0361,0.0151,5.5190,-0.1025,...,10.7504,19.4989,0.1944,1.2256,0.0167,1,0.0,-0.1129,0.00,Toxic


In [47]:
#a few functions to define information gain, a key aspect of splitting.
def InformationGain(varname, data):
    """
    InformationGain() takes in a variable name string on which to split, and the data on which
    to run the decision tree as a pandas dataframe. It returns the
    information gain for splitting on that variable, and for numerical variables,
    it returns the threshold on which it splits. 
    """
    
    #first, decide the split
    thres = GetThreshold(varname, data)
    
    #now what we are going to do is split based on this threshold and
    #calculate the entropy for each one.
    s1 = data[data[varname] >= thres]
    s2 = data[data[varname] < thres]
    v1 = list(s1["Class"])
    v2 = list(s2["Class"])
    v = list(data["Class"])
    
    n1 = len(v1)
    n2 = len(v2)
    n = len(v)
    #get entropies
    e1 = Entropy(v1)
    e2 = Entropy(v2)
    e = Entropy(v)

    #now we do a weighted average
    tot = e1 * (float(n1) / float(n)) + e2 * (float(n2) / float(n))
    gain = e - tot
    
    return gain, thres

def GetThreshold(varname, data):
    """
    GetThreshold() takes in a variable name string on which to split, and the data on which to split on.
    It returns the threshold of splitting for that variable. Note that this is a heuristic
    method that determines the split based on the mean between the minimum of one class and the
    maximum of the other outcome class; it may not give optimal split results.
    """
    #this is a heuristic method that will determine the split based on the mean
    #between the minimum value of one class and the maximum of another class.
    #it may not be good, but since we are testing on many variables, we are likely
    #to get a decent result.
    cat1 = data[data["Class"] == "Toxic"]
    cat2 = data[data["Class"] == "NonToxic"]

    vals1 = np.array(list(cat1[varname]))
    vals2 = np.array(list(cat2[varname]))
    if len(vals1) == 0:
        return np.min(vals2)
    if len(vals2) == 0:
        return np.min(vals1)

    min1 = np.min(vals1)
    max1 = np.max(vals1)
    min2 = np.min(vals2)
    max2 = np.max(vals2)

    #form split
    thres = (min1 + max2) / 2.0
    return thres

def Entropy(outcomes):
    """
    Entropy() takes a list of outcomes and returns the calculated conditional entropy.
    In the case of this project, the outcomes are all 'Toxic' or 'NonToxic'.
    """

    n = len(outcomes)

    #determine the number of toxic
    n_toxic = 0
    for i in range(n):
        if outcomes[i] == "Toxic":
            n_toxic += 1
    
    #nontoxic is easy from there
    n_nontoxic = n - n_toxic

    #and to do the rest we simply base off the entropy formula
    #but we need to check for zeroes too
    if n_toxic == 0 or n_nontoxic == 0:
        #we have zero entropy if we only have one class
        return 0
    
    f1 = float(n_toxic) / float(n)
    f2 = float(n_nontoxic) / float(n)

    p1 = -1 * f1 * np.log2(f1)
    p2 = -1 * f2 * np.log2(f2)
    e = p1 + p2

    return e

In [4]:
#a function to get the maximum information gain out of all variables. 
def GetBestSplit(data):
    vars = list(data.columns)
    n = len(vars)
    features = vars[:n-1] #removes the last column, the class.

    gain_list = []
    thres_list = []
    #loop through the features
    for f in features:
        if f != "Class": #a double check just to be sure
            currentGain, currentThres = InformationGain(f, data)
            gain_list.append(currentGain)
            thres_list.append(currentThres)

    best_index = np.argmax(np.array(gain_list))
    best_var = features[best_index]
    best_gain = gain_list[best_index]
    best_thres = thres_list[best_index]

    return best_var, best_gain, best_thres    

In [6]:
#initializing a node
def MakeNode(data, n):
    feature, gain, threshold = GetBestSplit(data)
    root = Node(n, feature, threshold, None)
    return root

#initializing a leaf
def MakeLeaf(data, n):
    numToxic = len(data[data["Class"] == "Toxic"])
    numNonToxic = len(data[data["Class"] == "NonToxic"])
    if numToxic > numNonToxic:
        leaf = Node(n, None, None, "Toxic")
    else:
        leaf = Node(n, None, None, "NonToxic")

    return leaf

In [52]:
#actual building of the decision tree

root = MakeNode(df, 1)
tree = BuildTree(root, df, 0, maxDepth=10)
#root.insert_left_child(df[df[root.var] < root.thres], 2)
#root.insert_right_child(df[df[root.var] >= root.thres], 3)
#rint(root)

In [51]:
#function to build the tree
def BuildTree(root, data, currentIter, min = 5, frac = 0.9, maxDepth = 20):
    """BuildTree() takes in the following inputs:
    - the root of the tree
    - the data for the tree, which is the full set at the beginning
    - the minimum number of items required to make a split
    - the minimum proportion of class to make a leaf
    - current iteration: set to 0 when first called.
    It returns the root of the tree, hopefully with all its branches extended.
    """
    #base case: we have less than the minimum number
    #or we have a fraction less than the threshold
    #in this case we make a leaf
    numToxic = len(data[data["Class"] == "Toxic"])
    numNonToxic = len(data[data["Class"] == "NonToxic"])
    prop_toxic = float(numToxic) / float(len(data))
    prop_nontox = float(numNonToxic) / float(len(data))
    if len(data) < min:
        root = MakeLeaf(data, root.id)
    elif prop_toxic > frac or prop_nontox > frac:
        root = MakeLeaf(data, root.id)
    elif currentIter >= maxDepth:
        root = MakeLeaf(data, root.id)
    else:
        #recursive case: we do a split
        #root = MakeNode(data, root.id)
        left_split = data[data[root.var] < root.thres]
        right_split = data[data[root.var] >= root.thres]

        root.insert_left_child(left_split, 2*root.id)
        root.c1 = BuildTree(root.c1, left_split.drop(columns = root.var), currentIter+1, min, frac, maxDepth)
        
        root.insert_right_child(right_split, 2*root.id+1)
        root.c2 = BuildTree(root.c2, right_split.drop(columns = root.var), currentIter+1 , min, frac, maxDepth) 

    return root 

In [53]:
print(tree)

Node 1: Split on piPC3 with threshold 5.2981 or return result None 
Left Child of 1: Node 2: Split on MATS1v with threshold 0.04835 or return result None 
Left Child of 2: Node 4: Split on None with threshold None or return result NonToxic 
Left Child of 4: None 
Right Child of 4: None 
Right Child of 2: Node 5: Split on SpMax8_Bhs with threshold 2.55375 or return result None 
Left Child of 5: Node 10: Split on None with threshold None or return result NonToxic 
Left Child of 10: None 
Right Child of 10: None 
Right Child of 5: Node 11: Split on None with threshold None or return result Toxic 
Left Child of 11: None 
Right Child of 11: None 
Right Child of 1: Node 3: Split on minHBint3 with threshold 0.29575 or return result None 
Left Child of 3: Node 6: Split on MDEN-33 with threshold 0.95515 or return result None 
Left Child of 6: Node 12: Split on R_TpiPCTPC with threshold 15.23025 or return result None 
Left Child of 12: Node 24: Split on ATS8m with threshold 11172.2219 or return 

In [61]:
def Predict(root, test_data):
    """
    Predict() is a function that uses the root of a pre-built tree and a pandas dataframe
    of testing data; these form the inputs. For each row in the dataframe, it
    traverses the tree and returns the result, storing it in a list of predicted
    outcomes.
    """
    outcomes = []
    for i in range(len(test_data)):
        current_point = test_data.iloc[i]
        current_outcome = PredictOne(root, current_point)
        outcomes.append(current_outcome)

    return outcomes

def PredictOne(root, dp):
    """ 
    PredictOne() is a subroutine of Predict() that takes in a single point and
    traverses the tree to predict it, returning the prediction.
    """
    #this is done through recursion.
    #base case: root is a leaf, in which case result != None
    if root.res != None:
        return root.res
    else:
        #recursive case; decide by threshold.
        current_val = dp[root.var]
        if current_val < root.thres:
            return PredictOne(root.c1, dp)
        else:
            return PredictOne(root.c2, dp)

In [66]:
def GetConfusionMatrix(data, pred):
    """ 
    GetMetrics() takes in a dataframe containing the true values of the predicted points
    in the column 'Class', and the list of predicted values given by the Predict() function.
    It returns the values for the confusion matrix in a list in the order:
    true pos, false pos, true neg, false neg.
    """
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    truth = list(data["Class"])

    #critically, we assume we get the same number of predictions.
    #if we don't, we may get an error, or not get the right values.
    if len(truth) != len(pred):
        print("Warning: Truth values and predicted values inequal length")
    for i in range(len(truth)):
        if truth[i] == pred[i]: #true prediction
            if truth[i] == "Toxic":
                tp += 1
            else:
                tn += 1
        else: #false prediction
            if truth[i] == "Toxic":
                fp += 1
            else:
                fn += 1
    
    return tp, fp, tn, fn


In [67]:
preds = Predict(tree, df)
GetConfusionMatrix(df, preds)

(48, 8, 113, 2)